<a href="https://colab.research.google.com/github/mohammadRahimi1993/BigData/blob/main/HomeWork2_Rahimi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 35.9 MB/s 


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [4]:
#reading dataset from my googledrive
!pip install gdown
!pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=4e2a6b3848264ed658cb5153f5819d7a09c3a240be3f4a7689d9e47122f53518
  Stored in directory: /tmp/pip-ephem-wheel-cache-6wej7b5n/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!unzip gdrive/My\ Drive/HW2_dataset.zip

Mounted at /content/gdrive
Archive:  gdrive/My Drive/HW2_dataset.zip
   creating: products_parquet/
  inflating: products_parquet/.part-00000-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00001-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00002-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00003-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00004-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00005-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00006-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00007-0f978a44-491b-411b-9bf5-266bd9d4e836-c000.snappy.parquet.crc  
  inflating: products_parquet/.part-00008-0f978a44-491b-411b

In [6]:
# Problem(1-1)
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName('SparkByExamples.com').getOrCreate()
Sales_DF=spark.read.parquet('/content/sales_parquet')
print("Number of sales is ")
Sales_DF.count()

Number of sales is 


20000040

In [7]:
# Problem(1-1)
Sellers_DF = spark.read.parquet('/content/sellers_parquet')
print("Number of sellers is ")
Sellers_DF.count()

Number of sellers is 


10

In [8]:
# Problem(1-1)
Product_DF = spark.read.parquet('/content/products_parquet')
print("Number of product is ")
Product_DF.count()

Number of product is 


75000000

In [9]:
# Problem(1-2)
spark.sql("CREATE TEMPORARY VIEW Saless USING parquet OPTIONS (path \"/content/sales_parquet\")")
Sales_DF.createOrReplaceTempView("Sales12")
spark.sql("SELECT COUNT(DISTINCT product_id) FROM Saless").show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                    993429|
+--------------------------+



In [10]:
# Problem(1-3)
# Sales_DF.show(3)
parkSQL= spark.sql("SELECT product_id, COUNT(product_id) FROM Saless GROUP BY product_id HAVING COUNT(product_id) >= 2")
parkSQL.show()

+----------+-----------------+
|product_id|count(product_id)|
+----------+-----------------+
|         0|         19000000|
|  32908362|                2|
|   1427289|                2|
|  26750979|                2|
|  45288884|                2|
|  68480776|                2|
|  71102646|                2|
|  71509129|                2|
|  24649333|                2|
|  49626228|                2|
|   1436058|                2|
|  61045775|                2|
|  13955602|                2|
|  61570737|                2|
|  43502792|                2|
|  68703653|                2|
|  35832907|                2|
|  62630575|                2|
|  71913021|                2|
|  59337762|                2|
+----------+-----------------+
only showing top 20 rows



In [11]:
#Problem_2
parkSQL= spark.sql("SELECT COUNT(DISTINCT product_id),date FROM Saless GROUP BY  date")
parkSQL.show()
parkSQL.count()

+--------------------------+----------+
|count(DISTINCT product_id)|      date|
+--------------------------+----------+
|                    100017|2020-07-03|
|                     99756|2020-07-07|
|                    100337|2020-07-01|
|                     99662|2020-07-08|
|                     99791|2020-07-04|
|                     98973|2020-07-10|
|                    100501|2020-07-09|
|                    100765|2020-07-06|
|                     99807|2020-07-02|
|                     99796|2020-07-05|
+--------------------------+----------+



10

In [12]:
#Problem_3
spark.sql("CREATE TEMPORARY VIEW PRODUCT USING parquet OPTIONS (path \"/content/products_parquet\")")
Result= spark.sql("SELECT s.product_id, s.num_pieces_sold*p.price AS total_price FROM Saless s JOIN PRODUCT p ON p.product_id=s.product_id")
Result.show(5)
from pyspark.sql.functions import col, sum,avg
Result.select(avg("total_price")).show()

+----------+-----------+
|product_id|total_price|
+----------+-----------+
|  10000047|      551.0|
|  10000715|     3825.0|
|  10002110|    12125.0|
|  10004929|     1394.0|
|  10005243|     4312.0|
+----------+-----------+
only showing top 5 rows

+------------------+
|  avg(total_price)|
+------------------+
|1246.1338560822878|
+------------------+



In [13]:
#Problem(4)
spark.sql("CREATE TEMPORARY VIEW Seller USING parquet OPTIONS (path \"/content/sellers_parquet\")")
result = spark.sql("SELECT * FROM Seller")
Result= spark.sql("SELECT ss.seller_id,ss.product_id, (ss.num_pieces_sold/(sr.daily_target*10)) AS sales_share FROM Saless ss JOIN Seller sr ON ss.seller_id=sr.seller_id" )
Result.groupBy("seller_id").avg().show()

+---------+--------------------+
|seller_id|    avg(sales_share)|
+---------+--------------------+
|        0|  2.0198858989476E-6|
|        7|2.595228787788167...|
|        3|1.628885370565929...|
|        8|9.213030375408889E-6|
|        5|4.211073965904027E-6|
|        6| 4.78214719436905E-6|
|        9|3.837913136180182E-6|
|        1|1.964233366461026...|
|        4|3.296428039825781...|
|        2|6.690408001060535E-6|
+---------+--------------------+



In [14]:
#Problem(5-1)
Result= spark.sql("SELECT s.seller_id,s.product_id, s.num_pieces_sold*p.price AS total_price FROM Saless s JOIN PRODUCT p ON p.product_id=s.product_id" )
totalSum=Result.groupBy("seller_id").sum().show()
Result.show(5)

+---------+----------------+
|seller_id|sum(total_price)|
+---------+----------------+
|        7|    4.23768474E8|
|        3|    4.25771873E8|
|        8|    4.21542107E8|
|        5|     4.2173518E8|
|        6|     4.2425561E8|
|        9|    4.26557098E8|
|        1|    4.23105788E8|
|        4|    4.23291612E8|
|        2|    4.24891581E8|
|        0| 2.1107807644E10|
+---------+----------------+

+---------+----------+-----------+
|seller_id|product_id|total_price|
+---------+----------+-----------+
|        9|  10000047|      551.0|
|        6|  10000715|     3825.0|
|        8|  10002110|    12125.0|
|        9|  10004929|     1394.0|
|        6|  10005243|     4312.0|
+---------+----------+-----------+
only showing top 5 rows



In [15]:
#Problem(5-2)
parkSQL= spark.sql("SELECT DISTINCT seller_id FROM Saless WHERE product_id = 0" )
parkSQL.show()

+---------+
|seller_id|
+---------+
|        0|
+---------+



In [16]:
#Problem_6

#Solution_1
from pyspark.sql.functions import md5, concat_ws, sha2, col, sha2, when

# Even
Sales_DF_EVEN = Sales_DF.filter( ( Sales_DF.order_id % 2) == 0)

Sales_DF_EVEN_Limited =Sales_DF_EVEN.limit(50)
for row in Sales_DF_EVEN_Limited.collect():
    for x in row['bill_raw_text']:
      if(x == 'A'):
        print(md5(row['bill_raw_text']))  
        
Sales_DF_EVEN_Limited.withColumn('hashed_bill', md5(col('bill_raw_text'))).show(5)

#Odd       
Sales_DF_ODD = Sales_DF.filter( ( Sales_DF.order_id % 2) == 1)
Sales_DF_ODD.withColumn("hashed_bill", sha2(col('bill_raw_text'), 256)).show(5)

#Solution_2
Sales_DF.withColumn('hashed_bill' , when(Sales_DF .order_id%2 ==0, md5('bill_raw_text')).otherwise(sha2(Sales_DF.bill_raw_text, 256))).show()

Column<'md5(yywjfihneygcvfnylxhvheoddmsbtdgicqzabenkzdpqpwfjwyuohebpkwvqoripstjkjzgteymjxvdptzuembynqhshpMwsgmxaplkezAgyozedsluwdotntgkkcwzcqflEuzwKyrrvnxjdpogpyaaxlymqukghqzybpfouhckccbqxisyxkvptwzbpniwzgjpmaszdbcaqlomravrjkqklkvopjnqxeyjudyvirxnjrohjnovsghcgiiphevwfxmdmeenlkbrddhfnuiyqveversqnkhespamyrphkfmgijarpgxkeaffuftoosivnuljrbddsptgkcebevpjhtekvrfgkjbfibvysjxcqlnbuxbzglxtkoskqwqpnqmpxdfownxhbvjqauwtodtyftptefumeapljbultxlldmqvooheuwhtvbxascgywjmagunqlxsrvscuqvdvzxynnlmsrrifumtxranqxojtnfrjgxwnjyqni)'>
Column<'md5(vqkfuzdwjvzzececenrprnvserfgcrnribresltkncnkvompugkgnseohbqvyehtwztmoqgdgtgyyaknwqusewynryluhjetnezkctkOxyuvimudrfdnngrakztquqckuyphdkywwzleFaRbxmjqwocrkxtrduoyonypbdlkcrzbqbphmynlwukvevipmtsnjvqukgwodjcbuahmqssrtjwgcotohklahtiqrcufsaqxsvtciflvwnhwldtgqjmeymqqfmhjaprazvrDexgoudeebtlvcnmiqyxIyjqdsknamngcmmsmpejtomxeujbtthbqdxuduqtjphgzaquoourgwjdxgmxspufitlaaascfwhrteevlqbqkacuoffumangvdmohyxookruvehxpqmdhjbfvfnqysxxyoajnlxbxglmooiicaeeMxbkblqbjqyqjjudybwvicmjzswtygaxg